In [1]:
import pandas as pd
import os
import numpy as np
from scipy.stats import norm

In [42]:
def bs_price(S,K,sigma,r,T, year_rate = 252, option_type = "call"):
    '''
    bs 期权定价 输出为期权价格
    输入格式：
    S:标的价格
    K:行权价
    sigma:波动率
    r:无风险利率
    T:年华剩余期限
    year_rate:年化系数
    '''

    T = T/year_rate
    d1 = (np.log(S/K) + (r + pow(sigma,2)/2)*T) / (sigma*np.sqrt(T)) 
    d2 = d1 - sigma*np.sqrt(T)
    if option_type == 'call':
        # return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        return np.maximum(S - K, 0) if T==0 else S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    elif option_type == 'put':
        # return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
        return np.maximum(K-S,0) if T==0 else K*np.exp(-r*T)*norm.cdf(-1*d2) - S*norm.cdf(-1*d1)
    else:
        print("期权类型错误")
        return -1

def bs_vega(S,K,sigma,r,T ,year_rate = 252):
    '''
    计算vega
    '''
    T = T/year_rate
    d1 = (np.log(S/K) + (r + pow(sigma,2)/2)*T) / (sigma*np.sqrt(np.abs(T)) )
    return S*norm.pdf(d1)*np.sqrt(np.abs(T))
    # return S * np.sqrt(abs(T)) * np.exp(-pow(d1,2)/2) / np.sqrt(2*np.pi)

def iv_bs_bisection(S, K, r, T, price, option_type, iv_uplimit = 1.0, iv_downlimit = 0.01, precision = 3, year_rate = 365, max_iterations = 200):
    '''二分法求iv'''
    left_iv, right_iv = iv_downlimit, iv_uplimit
    # left_price = BS_price(S,K,left_iv, r, T,  year_rate = year_rate, option_type = option_type)
    # right_price= BS_price(S,K,right_iv, r, T,  year_rate = year_rate, option_type = option_type)
    mid_iv = (left_iv + right_iv)/2
    mid_price = bs_price(S,K,mid_iv, r, T,  year_rate = year_rate, option_type = option_type)
    # print(f"first, the mid price is {mid_price}, the price is {price}")
    cnt = 0
    while abs(price - mid_price) >= 0.1**precision and cnt < max_iterations:
        if mid_price < price :
            left_iv = mid_iv
        else:
            right_iv = mid_iv
        mid_iv = (left_iv + right_iv)/2
        # if mid_iv < iv_uplimit or mid_iv > iv_downlimit:
        #     break
        # print(f"cnt = {cnt}, the left is{left_iv}, the right is{right_iv}, the mid is {mid_iv}")
        cnt += 1
        mid_price = bs_price(S,K,mid_iv, r, T,  year_rate = year_rate, option_type = option_type)
    return mid_iv

def find_vol_newton( S, K, r, T, target_value, option_type, start_sigma = 0.5, precision = 3, year_rate = 252, max_iterations = 100):
    '''迭代法求iv'''
    sigma = start_sigma
    for i in range(0, max_iterations):
        bs_price_ = bs_price(S,K,sigma,r,T, year_rate = year_rate, option_type = option_type)
        vega = bs_vega(S, K, T, r, sigma, year_rate = year_rate)*100
        diff = target_value - bs_price_  # our root
        if (abs(diff) < 0.1**precision):
            return sigma
        sigma = sigma + diff/(vega) # f(x) / f'(x)
    return sigma # value wasn't found, return best guess so far

def change(type_name):
    if type_name == 1:
        return "call"
    elif type_name == 0:
        return "put"
    else:
        return "wrong_type"

In [45]:
iv_bs_bisection(3.32, 3.4, 0.018,73, 0.33, 'call')

0.6083398558139801

In [46]:
find_vol_newton(3.32, 3.4, 0.018,73, 0.33, 'call')

0.6083396961968708

In [9]:
'''测试，数据清洗'''
csv_name = 'data/50etf_option_data_csv_cleaned/10000001.csv'
csv_out = 'data/out/10000001.csv'

count = 0
callcount = 0
putcount = 0

data = pd.read_csv(csv_name, encoding = 'utf-8', index_col= False)
data

,Unnamed: 0,date,pre_close,open,high,low,close,volume,amt,ptmtradeday,...,exercise_price,contract_unit,limit_month,listed_date,expire_date,exercise_date,settlement_date,reference_price,settle_mode,contract_state
0,0,2015/2/9 0:00,非数字,0.1820,0.2029,0.1699,0.1826,2501,4712265,28,...,2.2,10000,Mar-15,2015/2/9 0:00,2015/3/25 0:00,2015/3/25,2015/3/26,0.1812,实物资产,退市
1,1,2015/2/10 0:00,0.1826,0.1856,0.2144,0.1800,0.2072,1842,3744873,27,...,2.2,10000,Mar-15,2015/2/9 0:00,2015/3/25 0:00,2015/3/25,2015/3/26,0.1812,实物资产,退市
2,2,2015/2/11 0:00,0.2072,0.2083,0.2195,0.2028,0.2107,1999,4231181,26,...,2.2,10000,Mar-15,2015/2/9 0:00,2015/3/25 0:00,2015/3/25,2015/3/26,0.1812,实物资产,退市
3,3,2015/2/12 0:00,0.2107,0.2141,0.2143,0.1915,0.2109,2334,4804803,25,...,2.2,10000,Mar-15,2015/2/9 0:00,2015/3/25 0:00,2015/3/25,2015/3/26,0.1812,实物资产,退市
4,4,2015/2/13 0:00,0.2109,0.2130,0.2459,0.2090,0.2090,2066,4604396,24,...,2.2,10000,Mar-15,2015/2/9 0:00,2015/3/25 0:00,2015/3/25,2015/3/26,0.1812,实物资产,退市
5,5,2015/2/16 0:00,0.209,0.2111,0.2219,0.2066,0.2172,1153,2450037,23,...,2.2,10000,Mar-15,2015/2/9 0:00,2015/3/25 0:00,2015/3/25,2015/3/26,0.1812,实物资产,退市
6,6,2015/2/17 0:00,0.2172,0.2226,0.2408,0.2226,0.2296,759,1778307,22,...,2.2,10000,Mar-15,2015/2/9 0:00,2015/3/25 0:00,2015/3/25,2015/3/26,0.1812,实物资产,退市
7,7,2015/2/25 0:00,0.2296,0.2235,0.2250,0.1816,0.1885,815,1705437,21,...,2.2,10000,Mar-15,2015/2/9 0:00,2015/3/25 0:00,2015/3/25,2015/3/26,0.1812,实物资产,退市
8,8,2015/2/26 0:00,0.1885,0.1900,0.2530,0.1730,0.2530,1246,2747089,20,...,2.2,10000,Mar-15,2015/2/9 0:00,2015/3/25 0:00,2015/3/25,2015/3/26,0.1812,实物资产,退市
9,9,2015/2/27 0:00,0.253,0.2534,0.2739,0.2431,0.2488,873,2264841,19,...,2.2,10000,Mar-15,2015/2/9 0:00,2015/3/25 0:00,2015/3/25,2015/3/26,0.1812,实物资产,退市


In [21]:
tf50 = pd.read_excel(r'.\data\50etf_df.xlsx')
tf50.columns

Index(['Date', 'pre_close', 'open', 'high', 'low', 'close', 'volume', 'amt',
       'dealnum', 'chg', 'pct_chg', 'swing', 'turn'],
      dtype='object')

In [22]:
data.columns

Index(['date', 'ptmtradeday', 'exe_price', 'open', 'high', 'low', 'close',
       'call_or_put'],
      dtype='object')

In [11]:
data["date"] = pd.to_datetime(data["date"])
data = data[['date','ptmtradeday','exe_price', 'open', 'high','low','close', 'call_or_put']]
data.head()

,date,ptmtradeday,exe_price,open,high,low,close,call_or_put
0,2015-02-09,28,2.2,0.1820,0.2029,0.1699,0.1826,1
1,2015-02-10,27,2.2,0.1856,0.2144,0.1800,0.2072,1
2,2015-02-11,26,2.2,0.2083,0.2195,0.2028,0.2107,1
3,2015-02-12,25,2.2,0.2141,0.2143,0.1915,0.2109,1
4,2015-02-13,24,2.2,0.2130,0.2459,0.2090,0.2090,1


In [12]:
data['call_or_put'] = data['call_or_put'].apply(change)
data.head()


C:\fun\annaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,date,ptmtradeday,exe_price,open,high,low,close,call_or_put
0,2015-02-09,28,2.2,0.1820,0.2029,0.1699,0.1826,put
1,2015-02-10,27,2.2,0.1856,0.2144,0.1800,0.2072,put
2,2015-02-11,26,2.2,0.2083,0.2195,0.2028,0.2107,put
3,2015-02-12,25,2.2,0.2141,0.2143,0.1915,0.2109,put
4,2015-02-13,24,2.2,0.2130,0.2459,0.2090,0.2090,put


In [13]:
rate = pd.read_excel(r".\data\unrisked_rate.xlsx")
rate['date'] = pd.to_datetime(rate['date'])
rate['Value'] = rate['Value']/100
rate.head()

,date,Value
0,2012-10-08,0.035896
1,2012-10-09,0.035942
2,2012-10-10,0.036249
3,2012-10-11,0.036252
4,2012-10-12,0.036240


In [15]:
rate.shape

(2517, 2)

In [14]:
etf50 = pd.read_excel(r'.\data\50etf_df.xlsx')
etf50 = etf50[['Date','close']]
etf50['Date'] = pd.to_datetime(etf50['Date'])
etf50.head()

,Date,close
0,2005-02-23,0.876
1,2005-02-24,0.876
2,2005-02-25,0.880
3,2005-02-28,0.872
4,2005-03-01,0.867


In [16]:
etf50.shape

(4305, 2)

In [18]:
merged_option1 = pd.merge(rate, data, on = 'date', how = 'inner')
merged_option1

,date,Value,ptmtradeday,exe_price,open,high,low,close,call_or_put
0,2015-02-09,0.045769,28,2.2,0.1820,0.2029,0.1699,0.1826,put
1,2015-02-10,0.045696,27,2.2,0.1856,0.2144,0.1800,0.2072,put
2,2015-02-11,0.045715,26,2.2,0.2083,0.2195,0.2028,0.2107,put
3,2015-02-12,0.045540,25,2.2,0.2141,0.2143,0.1915,0.2109,put
4,2015-02-13,0.045669,24,2.2,0.2130,0.2459,0.2090,0.2090,put
5,2015-02-16,0.045716,23,2.2,0.2111,0.2219,0.2066,0.2172,put
6,2015-02-17,0.045626,22,2.2,0.2226,0.2408,0.2226,0.2296,put
7,2015-02-25,0.045924,21,2.2,0.2235,0.2250,0.1816,0.1885,put
8,2015-02-26,0.046345,20,2.2,0.1900,0.2530,0.1730,0.2530,put
9,2015-02-27,0.046482,19,2.2,0.2534,0.2739,0.2431,0.2488,put


In [20]:
merged_option2 = pd.merge(etf50, merged_option1, left_on = 'Date', right_on = 'date', how = 'inner')
merged_option2

,Date,close_x,date,Value,ptmtradeday,exe_price,open,high,low,close_y,call_or_put
0,2015-02-09,2.331,2015-02-09,0.045769,28,2.2,0.1820,0.2029,0.1699,0.1826,put
1,2015-02-10,2.369,2015-02-10,0.045696,27,2.2,0.1856,0.2144,0.1800,0.2072,put
2,2015-02-11,2.380,2015-02-11,0.045715,26,2.2,0.2083,0.2195,0.2028,0.2107,put
3,2015-02-12,2.384,2015-02-12,0.045540,25,2.2,0.2141,0.2143,0.1915,0.2109,put
4,2015-02-13,2.390,2015-02-13,0.045669,24,2.2,0.2130,0.2459,0.2090,0.2090,put
5,2015-02-16,2.394,2015-02-16,0.045716,23,2.2,0.2111,0.2219,0.2066,0.2172,put
6,2015-02-17,2.411,2015-02-17,0.045626,22,2.2,0.2226,0.2408,0.2226,0.2296,put
7,2015-02-25,2.370,2015-02-25,0.045924,21,2.2,0.2235,0.2250,0.1816,0.1885,put
8,2015-02-26,2.450,2015-02-26,0.046345,20,2.2,0.1900,0.2530,0.1730,0.2530,put
9,2015-02-27,2.438,2015-02-27,0.046482,19,2.2,0.2534,0.2739,0.2431,0.2488,put


In [23]:
merged_option2.columns

Index(['Date', 'close_x', 'date', 'Value', 'ptmtradeday', 'exe_price', 'open',
       'high', 'low', 'close_y', 'call_or_put'],
      dtype='object')

In [ ]:
merged_option2 = merged_option2[['call_or_put','date','ptmtradeday','exe_price','Value', 'open', 'high','low','close_y','close_x']]
merged_option2.columns = ['call_or_put', 'date','ptmtradeday','k', 'rate', 'open', 'high','low','close','s']
# print(merged_option2.s)
# merged_option2['iv'] = merged_option2.apply(lambda row : iv_bs_bisection(row['s'], row['k'], row['rate'], row['ptmtradeday'], row['close'], row['call_or_put'], iv_uplimit = 1.0, iv_downlimit = 0.0001, precision = 6, year_rate = 365),  axis=1)
merged_option2['iv'] = pd.Series
for i in range(merged_option2.shape[1]):
    merged_option2['iv'][i] = iv_bs_bisection(np.array(merged_option2['s'][i]), np.array(merged_option2['k'][i]), np.array(merged_option2['rate'][i]), np.array(merged_option2['ptmtradeday'][i]), np.array(merged_option2['close'][i]), np.array(merged_option2['call_or_put'][i]), iv_uplimit = 1.0, iv_downlimit = 0.0001, precision = 6, year_rate = 365, max_iterations= 120)
    # merged_option2['iv'][i] = find_vol_newton(merged_option2['s'][i],merged_option2['k'][i], merged_option2['rate'][i], merged_option2['ptmtradeday'][i], merged_option2['close'][i], merged_option2['call_or_put'][i],start_sigma = 0.1, precision = 3, year_rate = 365.0, max_iterations = 100)
    print(merged_option2['s'][i],merged_option2['k'][i], merged_option2['rate'][i])
    print(merged_option2['iv'][i])
    # break
merged_option2.to_csv(output_path)
